In [1]:
from CustomFeatures import MyFeatures
from NERmodel import NERmodel
import subprocess
import traceback, sys
import pickle

mf = MyFeatures()
print(mf.wordStructure(("Hell75o3,HA-niacidine0",)))
print(mf.wordStructure2(("Hell75o3,HA-niacidine0",)))
print(mf.wordStructure(("Hell75o_3,HA-niacidine0",)))

/home/pau/anaconda3/envs/ahltmai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Xx0x0X_x0
Xx0x0Xx0
Xx0x_0X_x0


In [1]:
# generate feature subsets
from CustomFeatures import MyFeatures
from NERmodel import NERmodel
import subprocess
import traceback, sys
import pickle


# name the batch training
training_set_name = "singleton20180516"

# preapre different MyFeature instances
fs = []
# number of model permutations
m = 1
# original model with all features
mf = MyFeatures()
fs.append(mf)
# for now it is random shuffling, but this is gonna be a manual selection of features
for i in range(m):
    try:
        mf = MyFeatures()
        mf.deriveNewFeatureSet(degree=3)
        fs.append(mf)
        #mf.printActiveFeatureFunctions()
        #print()
    except:
        pass
        
print("feature instances:")
print(fs)
print()    

# initialize
models=[]
for fset in fs:
    try:
        model = NERmodel(featureset=fset)
        model.setName(training_set_name)
        models.append(model)
    except:
        pass
print("models:")
print(models)
print()
    

# train all models
models_pickles =[None]*len(models)
print(len(models_pickles))
print(models_pickles)
for i in range(len(models)):
    model = models[i]
    try:
        model.trainFeatureExtraction("../data/LaboCase/Train", limit=None)
        model.saveTrainingFeatures("../data/models/multitrain-data"+str(i)+".json")
        model.newModelPipeline()
        models_pickles[i] = model.saveModelPipeline()
    except Exception:
        print("Exception in user code:",i)
        print("-"*60)
        traceback.print_exc(file=sys.stdout)
        print("-"*60)
        print("model "+str(i)+" has failed... ")
        models_pickles[i] = None
        
print("model pickles:")
print(models_pickles)
print()

# test all models
# PROBLEM I need the same feature set! so maybe the 
# pipeline should be pickled with the featureset
# so maybe we should use pickle of the whole NER?
results_files = [None]*len(models)
evaluation_files = [None]*len(models)
for i in range(len(models_pickles)):
    modelfile = models_pickles[i]
    fset = fs[i]
    try:
        model = NERmodel(featureset=fs[i])
        model.loadModelPipeline(filepath=modelfile)
        model.testFeatureExtraction(
            '../data/LaboCase/Test/Test for DrugNER task/DrugBank', 
            limit=None)
        model.predict()
        model.parsePredictionOutput('../data/models/task9.1_'+training_set_name+'_'+str(i)+'.txt')
        results_files[i] = '../data/models/multitrain-'+str(i)+'-output.csv'
        print("model "+str(i)+":")
        evaluation_files[i] = model.autoEvaluation()
    except  Exception:
        print("Exception in user code:",i)
        print("-"*60)
        traceback.print_exc(file=sys.stdout)
        print("-"*60)




/home/pau/anaconda3/envs/ahltmai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


feature instances:
[<CustomFeatures.MyFeatures object at 0x7fc688614128>, <CustomFeatures.MyFeatures object at 0x7fc641d19e48>]

models:
[<NERmodel.NERmodel object at 0x7fc641d19fd0>, <NERmodel.NERmodel object at 0x7fc642d40be0>]

2
[None, None]
model pickles:
['../data/models/svm-pipeline_20180516155814.pkl', '../data/models/svm-pipeline_20180516180415.pkl']

model 0:
0.0 
model 1:
0.0 


In [ ]:
print("model predictions:")
print(results_files)
print(evaluation_files)
print()

# print all result sets
for i in range(len(results_files)):
    file = results_files[i]
    file2 = evaluation_files[i]
    print("model "+ str(i)+ "-------------------------------------------------------------------")
    print(open(file,'r').read())
    print(open(file2,'r').read())
    print("-------------------------------------------------------------------")
    print()
    print()
    